In [20]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

# Define paths
train_dir = 'F:\\3rd_yr_project\\FER_transfer_learning\\FER_fear_dataset\\train'
test_dir = 'F:\\3rd_yr_project\\FER_transfer_learning\\FER_fear_dataset\\test'

# Preprocessing function to match MobileNetV3Large input requirements
def preprocess_input_fn(img):
    img = tf.image.resize(img, (224, 224))  # Resize to 224x224
    img = tf.keras.applications.mobilenet_v3.preprocess_input(img)  # MobileNetV3 preprocessing
    return img

# Data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_fn,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_fn)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Function to build the model
def build_model(hp):
    base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(hp.Int('units', min_value=32, max_value=512, step=32), activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)

    model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

Found 8138 images belonging to 2 classes.
Found 2031 images belonging to 2 classes.


In [22]:
# Hyperparameter tuning using Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_logs',
    project_name='mobilenetv3_fear_detection'
)


# Perform hyperparameter search
tuner.search(train_generator, epochs=10, validation_data=test_generator)

Trial 10 Complete [00h 39m 51s]
val_accuracy: 0.6312161684036255

Best val_accuracy So Far: 0.6312161684036255
Total elapsed time: 07h 09m 48s


In [24]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters
model = build_model(best_hps)

In [28]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator
)

Epoch 1/50
255/255 [==============================] - 320s 1s/step - loss: 0.5874 - accuracy: 0.6763 - val_loss: 0.6506 - val_accuracy: 0.6499
Epoch 2/50
255/255 [==============================] - 245s 962ms/step - loss: 0.5855 - accuracy: 0.6863 - val_loss: 0.6433 - val_accuracy: 0.6455
Epoch 3/50
255/255 [==============================] - 247s 968ms/step - loss: 0.5803 - accuracy: 0.6908 - val_loss: 0.6632 - val_accuracy: 0.6238
Epoch 4/50
255/255 [==============================] - 248s 973ms/step - loss: 0.5796 - accuracy: 0.6917 - val_loss: 0.6453 - val_accuracy: 0.6371
Epoch 5/50
255/255 [==============================] - 246s 965ms/step - loss: 0.5779 - accuracy: 0.6902 - val_loss: 0.6430 - val_accuracy: 0.6440
Epoch 6/50
255/255 [==============================] - 245s 963ms/step - loss: 0.5769 - accuracy: 0.6906 - val_loss: 0.6406 - val_accuracy: 0.6411
Epoch 7/50
255/255 [==============================] - 246s 964ms/step - loss: 0.5686 - accuracy: 0.6973 - val_loss: 0.6572 - va

In [29]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

64/64 [==============================] - 35s 541ms/step - loss: 0.6969 - accuracy: 0.6573
Test accuracy: 65.73%


In [27]:
# Save the model
model.save('fear_detection_mobilenetv3.h5')

F:\anaconda\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
